# Produce PDF report of OA stations showing RF waveforms

In [ ]:
# Imports
import os
# from collections import defaultdict

import numpy as np
import rf
import rf.imaging

In [ ]:
import importlib
import seismic.receiver_fn.rf_util as rf_util
import seismic.receiver_fn.rf_plot_utils as rf_plot_utils
import seismic.receiver_fn.rf_stacking as rf_stacking

In [ ]:
# importlib.reload(rf_plot_utils)

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

## Read source file

In [ ]:
rf_type = 'ZRT_td'

In [ ]:
network = 'OA'

In [ ]:
src_file = r"..\DATA\OA_rfs_20170911T000036-20181128T230620_{}_rev9_qual.h5".format(rf_type)

In [ ]:
data_all = rf_util.read_h5_rf(src_file)

In [ ]:
# data_all[0].stats.processing

In [ ]:
data_dict = rf_util.rf_to_dict(data_all)

## Summary page showing processing parameters

In [ ]:
i = 0
for st, station_db in data_dict.items():
    i += 1
    if i > 3:
        break
    channel = rf_util.choose_rf_source_channel(rf_type, station_db)
    channel_data = station_db[channel]
    rf_util.label_rf_quality_simple_amplitude(rf_type, channel_data)
    rf_stream = rf.RFStream([tr for tr in channel_data if tr.stats.predicted_quality == 'a']).sort(['back_azimuth'])

    # Find matching HHT channel data
    events = [tr.stats.event_id for tr in rf_stream]
    transverse_data = station_db['HHT']
    t_stream = rf.RFStream([tr for tr in transverse_data if tr.stats.event_id in events]).sort(['back_azimuth'])

    fig = rf_plot_utils.plot_rf_wheel([rf_stream, t_stream], layout=(1,2))
    plt.show()
    
    fig = rf_plot_utils.plot_rf_stack(rf_stream)
    plt.show()
    
    fig = rf_plot_utils.plot_rf_stack(t_stream)
    plt.show()

In [ ]:
with PdfPages('{}_RF_summary.pdf'.format(network)) as pdf:
    plt.rc('text', usetex=False)  # Would like to use Tex, but lack desktop PC privileges to update packages to what is required
    t_channel = 'HHT'
    for st, station_db in data_dict.items():
        # Choose RF channel
        channel = rf_util.choose_rf_source_channel(rf_type, station_db)
        channel_data = station_db[channel]

        # Label and filter quality
        rf_util.label_rf_quality_simple_amplitude(rf_type, channel_data)
        rf_stream = rf.RFStream([tr for tr in channel_data if tr.stats.predicted_quality == 'a']).sort(['back_azimuth'])

        # Find matching HHT channel data
        events = [tr.stats.event_id for tr in rf_stream]
        transverse_data = station_db['HHT']
        t_stream = rf.RFStream([tr for tr in transverse_data if tr.stats.event_id in events]).sort(['back_azimuth'])

        # Plot pinwheel of primary and transverse components
        fig = rf_plot_utils.plot_rf_wheel([rf_stream, t_stream], fontscaling=0.8)
        fig.set_size_inches(8.27, 11.69)
        plt.tight_layout()
        plt.subplots_adjust(hspace=0.15, top=0.95, bottom=0.15)
        ax = fig.gca()
        fig.text(-0.32, -0.32, "\n".join(rf_stream[0].stats.processing), fontsize=6, transform=ax.transAxes)
        pdf.savefig(dpi=300, papertype = 'a4', orientation = 'portrait')
        plt.close()

        # Plot RF stack of primary component
        fig = rf_plot_utils.plot_rf_stack(rf_stream)
        fig.set_size_inches(8.27, 11.69)
        fig.suptitle("Channel {}".format(rf_stream[0].stats.channel))
        pdf.savefig(dpi=300, papertype = 'a4', orientation = 'portrait')
        plt.close()

        # Plot RF stack of transverse component
        fig = rf_plot_utils.plot_rf_stack(t_stream)
        fig.set_size_inches(8.27, 11.69)
        fig.suptitle("Channel {}".format(t_stream[0].stats.channel))
        pdf.savefig(dpi=300, papertype = 'a4', orientation = 'portrait')
        plt.close()
    # end for
# end with